In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from io import StringIO

# Principle component analysis

#### 1) Loading trajectory

In [ ]:
def load_traj(filename):
    if filename.split('.')[-1] == 'npy':
        return np.load('trajectory_calpha.npy')
    else:
        with open(filename, "r") as f:
            docu = f.read()
        frames = docu.split("128\n generated by VMD\n  ")[1:]
        
        columns_names = ['type', 'x', 'y', 'z']
        dfs = [pd.read_csv(StringIO(f), names=columns_names, delim_whitespace=True, header=None) for f in frames]
        arrays = [df[['x','y','z']].values for df in dfs]
    return np.stack(arrays)

In [ ]:
traj = load_traj("trajectory_calpha.npy")
print(traj.shape)

#### 2) Mean-free trajectory

In [ ]:
mean_free_traj = traj - np.mean(traj, axis=0)

#### 3) Covariance matrix

For this section, we have to make a judgement call... A covariance matrix can be calculated for a matrix, but it doesn't make sense for rank 3 array. The question is... What are we trying to do principal component analysis on? 

The dimensions of the array represent timesteps, calpha atoms, xyz. I believe that it makes the most sense, given timeseries data to be looking for principal components about calpha atoms in time. We will create principal components of linear combiniations of calpha atoms across time. I.e. create three different matricies, each for xyz.

In [ ]:
# Example in x
x = mean_free_traj[..., 0].T

t_n = mean_free_traj.shape[0]
cov = (t_n - 1)**-1 * np.einsum('ik,jk', x, x)  # multiply rows, sum over columns.

# compare to cov
print(np.all(np.isclose(np.cov(x), cov)))

In [ ]:
# For all coordinates simultaneously.
xyz = [mean_free_traj[..., i].T for i in range(3)]
covs = [(t_n - 1)**-1 * np.einsum('ik,jk', i, i) for i in xyz]

# compare to cov
print([np.all(np.isclose(np.cov(i), cov)) for cov, i in zip(covs, xyz)])